In [1]:
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer, util

c:\Users\donchenko\Documents\mid_repos\notes_code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import spacy

# Загрузка крупной русскоязычной модели
nlp = spacy.load("ru_core_news_lg")


In [3]:

class CSVEmbeddingSearcher:
    def __init__(
        self,
        csv_path: str,
        embedding_column: str,
        model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    ):
        """
        :param csv_path: Путь к CSV-файлу.
        :param embedding_column: Название столбца, значение которого будем преобразовывать в вектор.
        :param model_name: Название модели Sentence Transformers.
        """
        self.csv_path = csv_path
        self.embedding_column = embedding_column
        self.model_name = model_name
        
        df = pd.read_csv(self.csv_path, sep=";", header=0)
        
        # 2) Оставляем только первые 5 столбцов (если их больше)
        if df.shape[1] > 5:
            df = df.iloc[:, :5]
        

        # 3) Удаляем строки, у которых в первой колонке (df.columns[0]) отсутствует значение (NaN)
        df.dropna(subset=[df.columns[0]], inplace=True)
        
        # (Опционально, если нужно также убрать случаи, когда 
        # значение в первой колонке — пустая строка)
        df = df[df[df.columns[0]].astype(str).str.strip() != ""]
        self.df = df
        
        
        # Загрузим модель (один раз)
        start_load = time.time()
        self.model = SentenceTransformer(self.model_name)
        self.model_load_time = time.time() - start_load
        
        # Создадим 6-й столбец с эмбеддингами
        self._create_embedding_column()
    
    def _create_embedding_column(self):
        """
        Приватный метод: проходится по датасету,
        кодирует значение из embedding_column в вектор
        и кладёт в новый столбец 'embedding_vector'.
        """
        if self.embedding_column not in self.df.columns:
            raise ValueError(f"Столбец '{self.embedding_column}' не найден в первых 5 колонках CSV.")
        
        embeddings_list = []
        start_time = time.time()
        
        # Кодируем каждую строку — для больших объёмов лучше использовать batch_size, но здесь для наглядности всё итеративно.
        for _, row in self.df.iterrows():
            text_value = str(row[self.embedding_column])  # на случай, если там не строка
            emb = self.model.encode(text_value, convert_to_numpy=True)
            embeddings_list.append(emb)
        
        self.df["embedding_vector"] = embeddings_list
        self.embedding_build_time = time.time() - start_time
    
    def search_nearest(self, query: str, top_k: int = 1):
        """
        Ищет в датафрейме ближайшие по косинусному сходству векторы (колонка 'embedding_vector')
        для заданного query. Возвращает top_k совпадений с их метаданными (индекс, похожесть).
        
        :param query: Текст для поиска ближайшего совпадения.
        :param top_k: Сколько ближайших совпадений вернуть.
        :return: Список словарей вида:
            [
              {
                "index": индекс_в_исходном_df,
                "similarity": сходство,
                "row_data": <запись датафрейма целиком>
              },
              ...
            ]
        """
        # Генерируем вектор для запроса
        query_emb = self.model.encode(query, convert_to_tensor=True)
        
        # Собираем все эмбеддинги из DataFrame в один массив
        all_emb = np.vstack(self.df["embedding_vector"].values)
        
        # Вычисляем косинусное сходство разом
        similarities = util.pytorch_cos_sim(query_emb, all_emb)[0].cpu().numpy()  # shape: [num_rows]
        
        # Отсортируем индексы по убыванию сходства
        sorted_indices = np.argsort(-similarities)
        
        # Возьмём top_k результатов
        top_indices = sorted_indices[:top_k]
        
        results = []
        for idx in top_indices:
            sim_score = float(similarities[idx])
            row_data = self.df.iloc[idx].to_dict()
            
            results.append({
                "index": idx,
                "similarity": sim_score,
                "Наименование": row_data["Наименование"],
                "Подкласс 3": row_data["Подкласс 3"],
            })
        
        return results
    
    def get_stats(self):
        """
        Возвращает простую статистику:
         - время загрузки модели,
         - время на создание эмбеддингов,
         - текущее количество строк и столбцов в датафрейме.
        """
        return {
            "model_name": self.model_name,
            "model_load_time": self.model_load_time,
            "embedding_build_time": getattr(self, "embedding_build_time", None),
            "data_shape": self.df.shape
        }



In [4]:
csv_path = "ОКС.csv"            
embed_col = "Наименование"      

# Создаём объект (модель загрузится, эмбеддинги построятся)
searcher = CSVEmbeddingSearcher(csv_path, embed_col)

In [ ]:
# Пример поиска
# query_text = "в зданиях детских дошкольных учреждений"
query_text = "Уклон наружных открытых лестниц, используемых для эвакуации из групповых ячеек, в зданиях детских дошкольных учреждений должен составлять не более 45° Ширину лестниц допускается выполнять не менее 0,8 м."
# query_text = "общественных зданиях"

top_k = 10
results = searcher.search_nearest(query_text, top_k=top_k)

print(f"\nТоп-{top_k} результатов поиска для запроса: '{query_text}'")
for res in results:
    print(f"- Индекс: {res['index']}, Сходство: {res['similarity']:.3f}, Наименование: {res['Наименование']}, KSI: {res['Подкласс 3']}")


Топ-10 результатов поиска для запроса: 'Уклон наружных открытых лестниц, используемых для эвакуации из групповых ячеек, в зданиях детских дошкольных учреждений должен составлять не более 45° Ширину лестниц допускается выполнять не менее 0,8 м.'
- Индекс: 671, Сходство: 0.606, Наименование: Здание школы-детского сада, KSI: KDBJ
- Индекс: 663, Сходство: 0.603, Наименование: Здание начальной школы, KSI: KDBA
- Индекс: 660, Сходство: 0.593, Наименование: Здание детского сада, KSI: KDAA
- Индекс: 560, Сходство: 0.587, Наименование: Сооружение подъемника (в том числе лифтового), KSI: HDCC
- Индекс: 466, Сходство: 0.582, Наименование: Здание детского дома, KSI: GCBA
- Индекс: 800, Сходство: 0.563, Наименование: Здание (сооружение) детского санатория, профилактория, KSI: RBAA
- Индекс: 661, Сходство: 0.560, Наименование: Здание детского сада с бассейном, KSI: KDAB
- Индекс: 667, Сходство: 0.558, Наименование: Здание школы-интерната, KSI: KDBE
- Индекс: 662, Сходство: 0.556, Наименование: Здан

In [12]:
def extract_nouns_with_modifiers(sentence: str):
    """
    Ищет существительные (NOUN), у которых зависимость в списке valid_deps,
    и собирает прилагательные, причастные обороты и т.п.
    Возвращает словарь, в котором ключ - существительное,
    значение - список строк (прилагательные / причастные обороты).
    """
    doc = nlp(sentence)
    result = {}
    
    # Какие зависимости считаем ключевыми (подлежащее, объект, генитивное дополнение, и т.д.)
    valid_deps = {"nsubj", "nsubj:pass", "obj", "iobj", "nmod"}
    
    for token in doc:
        # Берём только существительные, находящиеся в нужной зависимости
        if token.pos_ == "NOUN" and token.dep_ in valid_deps:
            # Соберём все модификаторы: классические amod (прилагательные) и acl (причастные обороты)
            modifiers = []
            for child in token.children:
                # Хотим захватить "наружных", "открытых" и "используемых для эвакуации"
                # 1) amod -> прилагательные, например "наружных", "открытых"
                # 2) acl или acl:relcl -> причастный оборот, например "используемых для эвакуации"
                # Также проверим морфологию: если это VERB и VerbForm=Part, то тоже считаем это причастием
                dep = child.dep_
                pos = child.pos_
                morph = child.morph
                
                # Условие, позволяющее «захватить» прилагательные и причастия
                if dep in ("amod", "acl", "acl:relcl"):
                    # Забираем всё поддерево, чтобы получить полный фрагмент
                    subtree_span = doc[child.left_edge.i : child.right_edge.i + 1]
                    modifiers.append(subtree_span.text)
                else:
                    # Если он не amod/acl, но это может быть причастие
                    # (например, pos_ == "VERB" и "VerbForm=Part" в морфологии)
                    if pos == "VERB" and "VerbForm=Part" in morph.get("VerbForm", []):
                        # Тоже берём всё поддерево
                        subtree_span = doc[child.left_edge.i : child.right_edge.i + 1]
                        modifiers.append(subtree_span.text)
            
            if modifiers:
                result[token.text] = modifiers
    
    return result

In [ ]:

data = extract_nouns_with_modifiers(query_text)

data

{'лестниц': ['наружных',
  'открытых',
  ', используемых для эвакуации из групповых ячеек,'],
 'ячеек': ['групповых'],
 'учреждений': ['детских', 'дошкольных']}

In [37]:
top_k = 5


for key, value in data.items():
    data_str = key + " " + " ".join(value)
    print(data_str)

    results = searcher.search_nearest(data_str, top_k=top_k)

    print(f"\nТоп-{top_k} результатов поиска для запроса: '{data_str}'")
    for res in results:
        print(f"- Индекс: {res['index']}, Сходство: {res['similarity']:.3f}, Наименование: {res['Наименование']}, KSI: {res['Подкласс 3']}")


    

лестниц наружных открытых , используемых для эвакуации из групповых ячеек,

Топ-5 результатов поиска для запроса: 'лестниц наружных открытых , используемых для эвакуации из групповых ячеек,'
- Индекс: 560, Сходство: 0.709, Наименование: Сооружение подъемника (в том числе лифтового), KSI: HDCC
- Индекс: 815, Сходство: 0.686, Наименование: Сооружение панорамного павильона, KSI: RCBD
- Индекс: 332, Сходство: 0.663, Наименование: Здание котельной, KSI: EAAB
- Индекс: 8, Сходство: 0.657, Наименование: Здание (сооружение) содержания молодняка, KSI: AADB
- Индекс: 9, Сходство: 0.656, Наименование: Здание птичника, KSI: AAEA
ячеек групповых

Топ-5 результатов поиска для запроса: 'ячеек групповых'
- Индекс: 826, Сходство: 0.744, Наименование: Конькобежный овал, KSI: RDCB
- Индекс: 603, Сходство: 0.705, Наименование: Совмещенная эстакада, KSI: HGAK
- Индекс: 828, Сходство: 0.697, Наименование: Велотрек, KSI: RDCD
- Индекс: 9, Сходство: 0.695, Наименование: Здание птичника, KSI: AAEA
- Индекс: 49